# Broadband Access

**Notebook: 2-api-data**

# Abstract

**Purpose:** 
This notebook (intended for a technical audience) pursues acquisition of custom data via calls to the Census Bureau’s API, with the goal of comparing broadband Internet access across lower-level geographies and demographics.

**Acknowledgments:** 
Data source: U.S. Census Bureau, 2019 American Community Survey 1-Year Estimates (https://data.census.gov/cedsci/table?q=broadband&g=0400000US51.050000&y=2019&d=ACS%201-Year%20Estimates%20Data%20Profiles&tid=ACSDP1Y2019.DP02&hidePreview=false)


# Scope (notebook)

 - obtain API key
 - import libraries
 - acquire custom api datasets
 - inspect and subset data
 - serialize objects for further study
 - 

# Setup

In [1]:
# importing packages
import os

In [2]:
# verifying the current working directory
os.getcwd()

'C:\\Users\\jamel\\myprojects\\acs-api\\notebooks'

In [3]:
# moving to the parent directory
os.chdir('..')

# verifying the current working directory
os.getcwd()

'C:\\Users\\jamel\\myprojects\\acs-api'

In [4]:
# loading python's `autoreload`, to update any external module changes
%load_ext autoreload

# turning-on `autoreload`
%autoreload 2

### Serialization

We will use the `pickle` serialization format to restore previously saved objects.

In [5]:
# importing the "helpers folder and contained modules as a package"
from helpers import *

Within `helpers` is the `helpers_func` package, which includes the custom `save_pickle`/`read_pickle` serialization/de-serialization modules.

In [6]:
# printing object-restoration message
print("Result of attempting to view the object after restoration:\n")

# restoring the data dictionary  and broadband data list from the serialized file
metadata_dict = read_pickle("pickles/metadata-dict.pkl")
broadband_geo_ids = read_pickle("pickles/broadband-geo-ids.pkl")

# verifying list restoration
broadband_geo_ids

Result of attempting to view the object after restoration:

Object restored from pickles/metadata-dict.pkl
Object restored from pickles/broadband-geo-ids.pkl


['DP02_0153E', 'DP02_0153M', 'DP02_0153PE', 'DP02_0153PM']

In [7]:
# # -----REFERENCE-----
# # serializing the data dictionary
# save_pickle(metadata_dict, "metadata-dict")

# # serializing the list of broadband variables
# save_pickle(broadband_geo_ids, "broadband-geo-ids")

### API

# Obtain

# Scrub 

# Explore

## Notebook Summary

...